In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
#!unzip "/content/drive/MyDrive/Parrot/img.zip" -d "./"

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import glob

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.utils import io

from packaging import version

import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [ ]:
import tensorflow as tf
print("TensorFlow version: {}".format(tf.__version__))
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

%load_ext tensorboard
from tensorboard.plugins.hparams import api as hp

gpu_list = tf.config.list_physical_devices('GPU')
print("GPUs Available:")
if gpu_list:
    try:
        for gpu in gpu_list:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpu_list), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print('None')


tf.keras.backend.clear_session()

from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print('Compute dtype: %s' % policy.compute_dtype)
print('Variable dtype: %s' % policy.variable_dtype)

debug_mode = False
tf.config.run_functions_eagerly(debug_mode)
if debug_mode:
    tf.data.experimental.enable_debug_mode()

print("Eager execution: {}".format(tf.executing_eagerly()))

!rm -rf ./tf_logs/ 

In [ ]:
data_path = '../data/img/'
train_ds = tf.data.Dataset.list_files(os.path.join(data_path, 'train', '*.png'))
test_ds = tf.data.Dataset.list_files(os.path.join(data_path, 'test', '*.png'))

def process_path(file_path):
    label = tf.strings.to_number(tf.strings.split(file_path, '_')[1])
    image = tf.io.decode_png(tf.io.read_file(file_path))
    return image, label


In [ ]:
%tensorboard --logdir ./tf_logs/hparam_tuning --host 0.0.0.0

In [ ]:
img_height = 9
img_width = 9
channels = 3
num_classes = 2

def get_model(base_filters, dense_filters, n_convs = 3, n_dense = 2):

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, channels)))
    m = 1
    for i in range(n_convs):
        model.add(tf.keras.layers.Conv2D(base_filters * m, 3, padding='same', activation='relu'))
        model.add(tf.keras.layers.BatchNormalization(axis=-1))
        model.add(tf.keras.layers.MaxPooling2D())
        m += m
    model.add(tf.keras.layers.Flatten())
    if n_dense > 1:
        m = 2 * (n_dense - 1)
    else:
        m = 1
    for i in range(n_dense):
        model.add(tf.keras.layers.Dense(dense_filters * m, activation='relu'))
        m = m // 2
    model.add(tf.keras.layers.Dense(num_classes))

    model.summary()

    return model

def get_optimizer(initial_lr):
    return mixed_precision.LossScaleOptimizer(tf.keras.optimizers.Adam(learning_rate=initial_lr))

def train(model, optimizer, epochs, batch_size, logdir,train_ds, test_ds):
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric_fn = tf.keras.metrics.CategoricalAccuracy()

    train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
    test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
    test_metric = tf.keras.metrics.Mean('test_metric', dtype=tf.float32)

    @tf.function()
    def _train_step(model, optimizer, _x_train, _y_train):
        with tf.GradientTape() as tape:
            predictions = model(_x_train, training=True)
            loss = loss_fn(_y_train, predictions)
            scaled_loss = optimizer.get_scaled_loss(loss)
        scaled_gradients = tape.gradient(scaled_loss, model.trainable_variables)
        gradients = optimizer.get_unscaled_gradients(scaled_gradients)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        train_loss(loss)

    @tf.function()
    def _test_step(model, _x_test, _y_test):
        predictions = model(_x_test, training=False)
        loss = loss_fn(_y_test, predictions)
        test_loss(loss)
        metric = metric_fn(tf.one_hot(tf.cast(_y_test, tf.int32), 2), predictions)
        test_metric(metric)

    log_dir = logdir

    summary_writer_train = tf.summary.create_file_writer(os.path.join(log_dir, 'Train'))
    summary_writer_test = tf.summary.create_file_writer(os.path.join(log_dir, 'Test'))
    progression_bar = tf.keras.utils.Progbar(target=epochs, verbose=1, unit_name='epoch',
                                                     stateful_metrics=['Train Loss'])

    train_ds = train_ds.map(process_path,
                         num_parallel_calls=tf.data.experimental.AUTOTUNE,
                         deterministic=False
                        ).batch(batch_size,
                                num_parallel_calls=tf.data.experimental.AUTOTUNE,
                                deterministic=False).prefetch(tf.data.experimental.AUTOTUNE).cache()

    test_ds = test_ds.map(process_path,
                         num_parallel_calls=tf.data.experimental.AUTOTUNE,
                         deterministic=True
                        ).batch(batch_size,
                                num_parallel_calls=tf.data.experimental.AUTOTUNE,
                                deterministic=True).prefetch(tf.data.experimental.AUTOTUNE).cache()

    for epoch in range(epochs):
        tf.summary.experimental.set_step(epoch)
        for (x_train, y_train) in train_ds:
            _train_step(model, optimizer, x_train, y_train)
        for (x_test, y_test) in test_ds:
            _test_step(model, x_test, y_test)
        with summary_writer_train.as_default():
            tf.summary.scalar('Loss', train_loss.result(), step=epoch)
        with summary_writer_test.as_default():
            tf.summary.scalar('Loss', test_loss.result(), step=epoch)
            tf.summary.scalar('Accuracy', test_metric.result(), step=epoch)
        progression_bar.update(current=epoch + 1,
                               values=[('Train Loss', train_loss.result()),
                                       ('Test Loss', test_loss.result()),
                                       ('Test Metric', test_metric.result())])
        train_loss.reset_states()
        test_loss.reset_states()
        test_metric.reset_states()


In [ ]:
session_num = 0
run_name = 'run'

HP_EPOCHS = hp.HParam('epochs', hp.Discrete([150]))
HP_BATCH = hp.HParam('batch', hp.Discrete([8]))
HP_LR = hp.HParam('initial_lr', hp.Discrete([1e-5]))
HP_FILTERS = hp.HParam('base_filters', hp.Discrete([128]))
HP_DENSEFILTERS = hp.HParam('dense_filters', hp.Discrete([8]))
HP_N_CONV = hp.HParam('n_conv_filters', hp.Discrete([1]))
HP_N_DENSE = hp.HParam('n_dense_filters', hp.Discrete([1]))

def hyper_parameters_config():
    for e in HP_EPOCHS.domain.values:
        for lr in HP_BATCH.domain.values:
            for a in HP_LR.domain.values:
                for bf in HP_FILTERS.domain.values:
                    for d in HP_DENSEFILTERS.domain.values:
                        for nc in HP_N_CONV.domain.values:
                            for nd in HP_N_DENSE.domain.values:
                                yield {
                                    HP_EPOCHS: e,
                                    HP_BATCH: lr,
                                    HP_LR: a,
                                    HP_FILTERS: bf,
                                    HP_DENSEFILTERS: d,
                                    HP_N_CONV: nc,
                                    HP_N_DENSE: nd,
                                }

for hparams in hyper_parameters_config():
    run_name = 'run-{}'.format(session_num)
    print('\n\n--- Starting trial: {}'.format(run_name))
    print({h.name: hparams[h] for h in hparams})

    run_logdir = 'tf_logs/hparam_tuning/' + run_name
    summary_writer = tf.summary.create_file_writer(run_logdir)

    with summary_writer.as_default():
        hp.hparams(hparams)
        model = get_model(hparams[HP_FILTERS], hparams[HP_DENSEFILTERS], hparams[HP_N_CONV], hparams[HP_N_DENSE])
        optimizer = get_optimizer(hparams[HP_LR])
        train(model, optimizer, hparams[HP_EPOCHS], hparams[HP_BATCH],
              run_logdir, train_ds = train_ds, test_ds=test_ds)
    session_num += 1
        